In [1]:
import urllib
from bs4 import BeautifulSoup
import csv
#import PyPDF2

In the Cell Above, I have declared all the necessary python libraries needed to scrape and clean the Data. It includes functionality such as selecting only the first page of the PDF since that is the only data we require, the libraries needed to scrape the data and select the desired columns using Beautiful Soup, and the abilitiy to write to a csv file so we can use it to visualize the data.

In [2]:
def PDF_Page_Selector(aFileName):
    '''
    This Function takes one input: an Input File in pdf format, it then only selects the first page and outputs a
    new pdf only containing that first page
    '''
    assert aFileName[-4:] == ".pdf"
    thePDFReader = PyPDF2.PdfFileReader(open(aFileName, "rb"))
    theFirstPage = thePDFReader.getPage(0)  # extracting the first page since it contains the table we desire
    theWriter = PyPDF2.PdfFileWriter()
    theWriter.addPage(theFirstPage)
    with open("TheArkansasTable.pdf", "wb") as theOutputStream: #writing the first page to new PDF file
        theWriter.write(theOutputStream)

#PDF_Page_Selector('NSDUHsaeArkansas2016.pdf') #input file name here

In the Module above, I ran it in Pycharm because I had troubles importing PyPDF2 into jupyterhub. The function above takes one input arguement, the file name of the file we would like to snip a page from, and creates a new file that only contains the page we desire. ex) [The Input File](NSDUHsaeCalifornia2016.pdf) and [The Output File](TheCaliforniaTable.pdf)

In [3]:
def Scraping_HTML_File(aFileName):
    '''
    This function takes an input html file, scrapes the data we need and cleans the numbers, commas, and unnecessary 
    data out
    '''
    assert aFileName[-5:] == ".html"
    theRowData = []
    thePage = urllib.urlopen(aFileName)
    theSoup = BeautifulSoup(thePage, 'html.parser')
    #theDataHeader is the header and theData is the actual data
    theDataHeader = theSoup.find('div', attrs = {'style' : 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:538px; top:143px; width:20px; height:150px;'})
    theHeaderText = theDataHeader.text
    theData = theSoup.find('div', attrs = {'style' : 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:538px; top:366px; width:20px; height:142px;'})
    theDataText = theData.text
    #theCategoryHeader is the category header and the categories correspond to the data in the row
    theCategoryHeader = theSoup.find('div', attrs = {'style' : 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:39px; top:143px; width:63px; height:22px;'})
    theCategoryHeaderText = theCategoryHeader.text
    theCategories = theSoup.find('div', attrs = {'style' : 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:49px; top:461px; width:113px; height:47px;'})
    #stripping all numbers and commas from the string
    theCategoriesText = (''.join([theChar for theChar in theCategories.text if not (theChar.isdigit())])).replace(',','')
    
    print(theCategoryHeaderText[:7])
    print(theCategoriesText)
    print(theHeaderText[:3])
    print(theDataText[41:])
    
    theCombinedCategoryText = theCategoryHeaderText[:7] + '\n' + theCategoriesText
    theCombinedData = theHeaderText[:3] + '\n' + theDataText[41:]
    for theCategoryLine, theDataLine in zip(theCombinedCategoryText.splitlines(), theCombinedData.splitlines()):
        theRowData.append((theCategoryLine,theDataLine))
        
    with open(aFileName[:-5] + '.csv', 'a') as theOutputStream: 
        theWriter = csv.writer(theOutputStream)
        for theCategory, theIndData in theRowData:
            theWriter.writerow([theCategory, theIndData])
     
    with open(aFileName[:-5] + '.csv') as theInputStream:
        theReader = csv.reader(theInputStream)
        for theRow in theReader:
            print(theRow)
    
Scraping_HTML_File('TheNevadaTable.html')

Measure
Serious Mental Illness
Any Mental Illness
Received Mental Health Services 
Had Serious Thoughts of Suicide
Major Depressive Episode 

18+
93 
404 
256 
95 
162 

['Measure', '18+']
['Serious Mental Illness', '93 ']
['Any Mental Illness', '404 ']
['Received Mental Health Services ', '256 ']
['Had Serious Thoughts of Suicide', '95 ']
['Major Depressive Episode ', '162 ']


The Function above takes an input html file, reads the data, selects the data required, and outputs it into a new csv file. I first used pdf2txt commandline command after calling the first function so the pdf dataset becomes a html file that I could scrape. I then initialize an empty list, theRowData, to which I will append individual row data in afterwards. 

[Example of an HTML file](TheCaliforniaTable.html) (disclaimer: IDK why it is not showing the html tags, but it should be similar to the Example of a Tag I look for)

The next step I took was to create a beautiful soup of the html file so I could read the individual HTML tags. I then looked for the Category Header Tag, which is Measure in the above case, the categories, which are Serious Mental Illness, Any Mental Illness, Received Mental Health Services, Had Serious Thoughts of Suicide, and Major Depressive Episode, the Data Header, which is 18+, and lastly, the data itself, which in this instance are 93, 404, 256, 95, and 162. 
[Example of a Tag I look for](Div_Tag_Snip.png)

All of the scraped data except for the categories is then cleaned by only selecting the data we want by string splicing, with each state having a different theDataText splicing and the other ones stay consistent.

The Categories Text's numbers and commas are then filtered because we do not need them. Later, I combined the category header with the categories text by joining the with a new line in between and I undergo the same process for the data header and the data.

Lastly, theRowData list is iterated through and each row is written into a new csv file that has the same name as the input filename, but a different file format. 

[Example of an Output CSV File](TheCaliforniaTable.csv)